In [1]:
import requests
import json
import pandas as pd

In [2]:
headers = {'Content-type': 'application/json'}
data = json.dumps({"seriesid": [
    'CUUR0000SA0'
    ]
        ,"startyear":"2012", "endyear":"2022", "registrationkey": "6a4fc3836e344229b9fe9dc5925c842e",
       "catalog": "true",})
p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
json_data = json.loads(p.text)

## Catalog data - Info that pertains to the Series ID

In [3]:
json_data['Results']['series'][0]['catalog']

{'series_title': 'All items in U.S. city average, all urban consumers, not seasonally adjusted',
 'series_id': 'CUUR0000SA0',
 'seasonality': 'Not Seasonally Adjusted',
 'survey_name': 'CPI for All Urban Consumers (CPI-U)',
 'survey_abbreviation': 'CU',
 'measure_data_type': 'All items',
 'area': 'U.S. city average',
 'item': 'All items'}

## Data Points - Specific data points within the series id

In [4]:
json_data['Results']['series'][0]['data'][0]

{'year': '2022',
 'period': 'M11',
 'periodName': 'November',
 'latest': 'true',
 'value': '297.711',
 'footnotes': [{}]}

In [5]:
json_data['Results']['series'][0]['catalog']

{'series_title': 'All items in U.S. city average, all urban consumers, not seasonally adjusted',
 'series_id': 'CUUR0000SA0',
 'seasonality': 'Not Seasonally Adjusted',
 'survey_name': 'CPI for All Urban Consumers (CPI-U)',
 'survey_abbreviation': 'CU',
 'measure_data_type': 'All items',
 'area': 'U.S. city average',
 'item': 'All items'}

In [6]:
df = pd.DataFrame(columns=[
    'series_title', 'series_id', 'seasonality', 'survey_name', 'survey_abbreviation', 'measure_data_type', 'area'
    , 'year', 'period', 'period_name', 'value'
                          ])

# KeyErrors
### I received a KeyError on the 'calculations' key and some other keys, which after some digging I figured out was the result of differences in fields and keys between the different series. Because of this, I decided to split up my API calls by category (each category has generally the same key fields for the fields I need).

In [7]:
for series in json_data['Results']['series']:
    series_title = series['catalog']['series_title']
    series_id = series['catalog']['series_id']
    seasonality = series['catalog']['seasonality']
    survey_name = series['catalog']['survey_name']
    survey_abbreviation = series['catalog']['survey_abbreviation']
    measure_data_type = series['catalog']['measure_data_type']
    area = series['catalog']['area']
    for data in series['data']:
        year = data['year']
        period = data['period']
        period_name = data['periodName']
        value = data['value']
        
        #save data in dataframe
        df = pd.concat([df, pd.DataFrame.from_records([{
            'series_title': series_title
            , 'series_id': series_id
            , 'seasonality': seasonality
            , 'survey_name': survey_name
            , 'survey_abbreviation': survey_abbreviation
            , 'measure_data_type': measure_data_type
            , 'area': area
            , 'year': year
            , 'period': period
            , 'period_name': period_name
            , 'value': value
        }])], ignore_index=True)

In [8]:
for series in json_data['Results']['series']:
    print(series['catalog'])

{'series_title': 'All items in U.S. city average, all urban consumers, not seasonally adjusted', 'series_id': 'CUUR0000SA0', 'seasonality': 'Not Seasonally Adjusted', 'survey_name': 'CPI for All Urban Consumers (CPI-U)', 'survey_abbreviation': 'CU', 'measure_data_type': 'All items', 'area': 'U.S. city average', 'item': 'All items'}


In [9]:
print(df)

                                          series_title    series_id  \
0    All items in U.S. city average, all urban cons...  CUUR0000SA0   
1    All items in U.S. city average, all urban cons...  CUUR0000SA0   
2    All items in U.S. city average, all urban cons...  CUUR0000SA0   
3    All items in U.S. city average, all urban cons...  CUUR0000SA0   
4    All items in U.S. city average, all urban cons...  CUUR0000SA0   
..                                                 ...          ...   
126  All items in U.S. city average, all urban cons...  CUUR0000SA0   
127  All items in U.S. city average, all urban cons...  CUUR0000SA0   
128  All items in U.S. city average, all urban cons...  CUUR0000SA0   
129  All items in U.S. city average, all urban cons...  CUUR0000SA0   
130  All items in U.S. city average, all urban cons...  CUUR0000SA0   

                 seasonality                          survey_name  \
0    Not Seasonally Adjusted  CPI for All Urban Consumers (CPI-U)   
1    Not 

In [10]:
#df.to_excel("C:/Users/grantarbuckle/Desktop/Python Practice/BLS_Compensation_Data.xlsx")